<!-- PROFILE LINK -->

<h1 style = "text-align:center; font-size: 30px">PROGRAMACIÓN CONCURRENTE Y DISTRIBUIDA</h1>
<a href ="https://github.com/christianyang22">
<img src="https://avatars.githubusercontent.com/u/103989159?s=400&u=52cbb863b4a897985f0548922e6e678f9c98cc1c&v=4" style="float:left;vertical-align:centre" width="110" height="110" title = "Christian Jonathan Yang Gonzalez 22133089">
<a href ="https://github.com/christianyang22?tab=repositories">
<img src="https://media-exp1.licdn.com/dms/image/C4D0BAQEu_Aa76fcmPw/company-logo_200_200/0/1630998679429?e=2159024400&v=beta&t=2feC9GG7RWCWizTqsK5HzDWB1TL2DOQuAVwnMf6FFvk" title = "M421" alt="UE" width="110" height="110" style="float:right;vertical-align:centre">

# Memoria

## Introduccion

En este trabajo aprenderemos como enlazarnos a un servidor y poder realizar un chatroom, con la finalidad de entender como trabajan los hilos y el socket.

Objetivos:
    
    1.  Conocer los problemas que genera la concurrencia y que trataremos a lo largo de la asignatura.  
    2.  Aplicar las posibles soluciones a estos problemas y los mecanismos que se utilizan para resolverlos mediante el uso de hilos.  
    3.  Nombre de la actividad AI1  
    
Tendremos que modificar un programa dado por el profesor, con las siguientes criterios:
    
    a.  Deberá instanciar 6 clientes y cada uno de ellos debera ingresar mediante el teclado la dirección ip y el puerto, que será los últimos 5 dígitos de su número de expediente. 2 pto.  
    b.  Que cada vez que un cliente se conecte, este pueda y deba elegir un nickname que será vuestro correo de la universidad y que deberá ser mostrado cada vez que escriba un texto. 2 pto.  
    c.  Que el servidor, imprima y muestre la lista de usuarios (nicknames) conectados al mismo (en ese instante). 2pto.  
    d.  Que el servidor almacene el historial de chat de todos los usuarios conectados durante la sesión y lo guarde en un archivo que se deberá llamar uvuestroExp.txt (u21535220AI1.txt) 4pto.

## Cliente

### Librerias

Gracias a estas librerias podremos realizar el trabajo de manipulacion de hilos, enlace con servidor-cliente, cifrado/descifrado de mensajes y funcionalidades del sistema operativo

In [ ]:
import threading #Libreria threading, para poner matipular los hilos
import sys #Libreria de sistema, para la interaccion con el interprete (poder hablar con el sistema)
import socket #Libreria socket, para poder enlazarnos cliente-servidoer
import pickle #Libreria pickle para la encriptacion del codigo
import os #Libreria del sistema operativo, para el uso de funciones dependiente del sistema operativo.

### Contructor principal (__init__)

1. Aqui como primera instancia declaramos y inicializamos por pantalla los parametros del contructor host(ip), port (puerto) y user(nombre del usuario)
2. Declaramos y inicializamos self.s para poder utilizar la librerias de socker 
3. El bucle while del user para que si o si te tenga que dar un nombre, ya que es un requisito primordial para el ChatRoom para poder saber quien eres en el chat, no sale de el hasta que el string sea no vacio y lo guardamos en la variable self.username.
4. Utilizamos la funcion open() para poder crear el txt de la listas de usuarios para poder imprimir al servidor la lista de usuarios activos en el chat, utilizamos modo append, para que no se sobre escriba la lista.
5. Creamos la tupla de la variable self.s para poder guardar al usuario y el puerto, para conectarse con el servidor.
6. Un print por pantalla el proceso actual el total de hilos que estan disponibles para ti, luego el hilo instanciado para ti para intercambiar la informacion con el servidor en el threading.thread().star(). 

In [ ]:
class Cliente():
    #Constructor principal
	def __init__(self, host=input("Introduzca la IP del servidor? "), port=int(input("Introduzca el puerto del servidor? ")),user=""): #Los parametros del constructor princial Host(ip), port(puerto) y user(usuario).
		self.s = socket.socket() #inicializar la variable para poder utilizar las funciones de la libreria socket
		while(user==""): #Bucle while para pedir el usuario hasta que le pases uno no vacio.
			user=input("Nombre del usuario: ") #pide por teclado que le pases un usuario
		self.username = user #inicializa y guarda el usuario que le has pasado
		with open("u22133089AI1UserList.txt", "a") as f: #Funcion para crear un .txt para crear una lista de usuarios, que se conectan
			f.write(self.username + "\n") #Pasa al txt el usuario escogido, apila y lo guarda
		self.s.connect((host, int(port))) #Le pasas la ip y el puerto en la que esta instanciado el servidor
		print('\n\tProceso con PID = ',os.getpid(), '\n\tHilo PRINCIPAL con ID =',threading.currentThread().getName(), '\n\tHilo en modo DAEMON = ', threading.currentThread().isDaemon(),'\n\tTotal Hilos activos en este punto del programa =', threading.active_count()) #Te pasa proceso actual, como el hilo encargado de tu chat y lo hace demonio, los hilos activos para todo esto.
		threading.Thread(target=self.recibir, daemon=True).start() #Instancia el hilo del chat

### Bucle para salida del chat

En este bucle y su contenido se carga de estar revisando lo que le pasa el usuario por teclado, si es un "0", significa que quiere abandonar el chat y si no llama a la funcion enviar, para cifrar el mensaje y mandarlo al servidor, se mantiene siempre activo a la espera de lo que escribe por teclado el usuario.

In [ ]:
		while True: #Bucle para salir del programa
			msg = input('\nEscriba texto ?   ** Enviar = ENTER   ** Salir Chat = 0 \n') #muestra por pantalla la forma de salir de el, para que sepas que caracter clave es.
			if msg != '0' : self.enviar(msg) #Si no es el caracter clave para salir lo manda a la funcion enviar para mandarlo al servidor y lo transmita a los demas chat.
			else: #si es =0
				print(" **** Me piro vampiro; cierro socket y mato al CLIENTE con PID = ", os.getpid()) #te dice que termina el proceso y cierra el programa
				self.mantenerUsariosActivos(self.username) #Elimina de la lista tu usuario, para que solo aparezcan los usuarios activos.
				self.s.close() #Cierra el socket
				sys.exit() #Sale del programa y cierra interracion del programa por terminal.

### Usuarios activos (archivo txt)

Este metodo se encarga principalmente de mantener los usuarios activos, eso quiere decir que si un usuario deja el chat se encarga de revisar el txt y sobreescribirlo con los usuarios activos.

In [ ]:
	def mantenerUsariosActivos(self, user): #Metodo para eliminar un nombre del txt, donde se guardan todos los user, cuando se salga el usuario del chat
		userNamesList = [] #declarar un array para contener los usuarios de cada linea dentro de ella.
		with open("u22133089AI1UserList.txt", "r") as f: #abrimos el txt, para leerlo
					username = f.readlines() #Le damos el vamor de leer el txt que abrimos anterior mente
					for n in username: #Introducimos esa lista en el for, para que recorra todos los usuarios
						if ((user + "\n") != n): #Si el nickname no se encuentra en ella 
							userNamesList.append(n) #Se agrega al final del array añadiendo un nuevo valor
		with open("u22133089AI1UserList.txt", "w") as f: #abrimos el txt para sobreescribirlo
			for n in userNamesList: #Pasamos la lista al bucle para recorrerla
				f.write(n) #Sobre escribimos la lista de usuarios para mantener solo los activos.

### Recibir mensajes

Este metodo se encarga de descencriptar los mensajes que manda el servidor y imprimirlos por pantalla para que el usuario pueda recibir la informacion del chat, siempre esta activo para recibir mensaje en cualquier momento.

In [ ]:
	def recibir(self): #Metodo que recibe los mensajes que manda el servidor
		print('\nHilo RECIBIR con ID =',threading.currentThread().getName(), '\n\tPertenece al PROCESO con PID', os.getpid(), "\n\tHilos activos TOTALES ", threading.active_count()) #imprime que hilos estas utilizando y que id y proceso de cual esta encargado. 
		while True: #Bucle que esta siempre activo para recibir mensajes
			try: #Try except, para que no se pare el programa si llega haber un error o error fatal.
				data = self.s.recv(1024) #Guardamos en una variable el mensaje cifrado y se desifra, mediante la otra variable que esta ligada con la libreria socket (self.s.recv(1024)), ademas de la cantidad de bits que puede tener el mensaje 
				if data: print(pickle.loads(data)) #Condicional para data que imprima el mensaje por pantalla
			except: pass

### Enviar mensajes

Este metodo se ejecuta cuando el constructor principal __init__ lo esta llamando, ya que lo que recibio por teclado es algo diferente de 0, eso significa que quiere mandar un mensaje a todos los demas usuarios, por lo que el mensaje se envia al servidor, para que el se encargue de transmitirlo a los demas. 

In [ ]:
	def enviar(self, msg): #Metodo enviar, para mandar los mensaje al servidor los pase a los demas usuarios.
		self.s.send(pickle.dumps(self.username + ": " + msg)) #Serializar, mediante la libreria pickle y mandarlo mediante el enlaze del socket (self.s.send)
		with open("u22133089AI1.txt", "a") as f: #Creamos un txt, donde se guardara un historial de los mensajes que manda el cliente, modo append, para que no se sobreescriba y se apilen los datos que se ingresen.
			f.write(self.username + ": " + msg + "\n") #Funcion escribir rellena el txt con el historial.

### Variable que ejecuta el programa

Se declara y inicializa la variable que ejecuta la clase cliente (programa).

In [ ]:
arrancar = Cliente() #Ejecuta el programa.

## Servidor

### Librerias

Gracias a estas librerias podremos realizar el trabajo de manipulacion de hilos, enlace con servidor-cliente, cifrado/descifrado de mensajes y funcionalidades del sistema operativo

In [ ]:
import threading #Libreria threading, para poner matipular los hilos
import sys #Libreria de sistema, para la interaccion con el interprete (poder hablar con el sistema)
import socket #Libreria socket, para poder enlazarnos cliente-servidoer
import pickle #Libreria pickle para la encriptacion del codigo
import os #Libreria del sistema operativo, para el uso de funciones dependiente del sistema operativo.

### Contructor principal (__init__)

1. Aqui como primera instancia declaramos y inicializamos por pantalla los parametros del contructor host(ip), port (puerto).
2. Declaramos el array clientes, que contendra los puertos de los clientes conectados al servidor.
3. Imprime por pantalla el ip al cual estas conectado y el proceso, hilos y todo lo relacionado al proceso.
4. Declaramos y inicializamos self.s para poder utilizar la librerias de socker 
5. self.s.bind() esta funcion de la libreria nos permite enlazarnos con el address (ip), si el puerto esta ocupado no enciende el servidor, hasta que le pase un puerto libre para poder instanciarlo.
4. Cuantos usuarios podran solicitarse conectar al servidor.
5. Que socket no se bloquee, mientras se ejecuta el programa.
6. Hace el hilo utilizados en los metodos aceptar y procesar sean demonios, para que esten siempre activos.

In [ ]:
class Servidor():
    # Constructor principal.
	def __init__(self, host=socket.gethostname(), port=int(input("Que puerto quiere usar ? "))): #Parametros del constructor host(ip), port(puerto).
		self.clientes = [] #Array para guardar el puerto por el que ha ingresado.
		print('\nSu IP actual es : ',socket.gethostbyname(host)) #Imprime por pantalla para que semas cual es tu ip.
		print('\n\tProceso con PID = ',os.getpid(), '\n\tHilo PRINCIPAL con ID =',threading.currentThread().getName(), '\n\tHilo en modo DAEMON = ', threading.currentThread().isDaemon(), '\n\tTotal Hilos activos en este punto del programa =', threading.active_count()) #Te pasa proceso actual, como el hilo encargado de tu chat y lo hace demonio, los hilos activos para todo esto.
		self.s = socket.socket() #inicializar la variable para poder utilizar las funciones de la libreria socket
		self.s.bind((str(host), int(port))) #Enlaza el socket al address, tiene que estar libre el socket, si no no podra enlazarse y encender el servidor.
		self.s.listen(30) #Decirle al socket cuantos usuarios podran solicitarse conectar al servidor.
		self.s.setblocking(False) #Para que que el socket sea no bloqueante, para que no se bloque durante la ejecucion del programa.

		threading.Thread(target=self.aceptarC, daemon=True).start() #Hace que este hilo sea un hilo demonio mientras se ejecute el programa.
		threading.Thread(target=self.procesarC, daemon=True).start() #Hace que este hilo sea un hilo demonio mientras se ejecute el programa.

### Bucle para salida del chat

En este bucle y su contenido se carga de estar revisando lo que le pasa el usuario por teclado, si es un "0", significa que quiere abandonar el chat comenzamos a cerrar todo el cervidor, dejamos en blanco el listado de usuarios que se conectaron (si el servidor se cae, no tiene que haber usuarios activos) sobreescribimos con un archivo en blanco, ademas de cerrar el socket y el interprete.

In [ ]:
		while True: #Bucle para salir del programa
			msg = input('\n << SALIR = 0 >> \n') #Te dice por pantalla como cerrar el servidor.
			if msg == '0': #El mensaje recibido es igual a "0", entra.
				print(" **** Me piro vampiro; cierro socket y mato SERVER con PID = ", os.getpid()) #Imprime que se cerrara el server y el proceso que se termina
				with open("u22133089AI1UserList.txt", "w") as f: #Resetea la lista de usuario activos en blanco.
					f.write(" ") #Guarda un archivo en blanco.
				self.s.close() #Cierra el socket.
				sys.exit() #Cierra el interprete
			else: pass #Si no es un 0 pasa del tema y sigue el programa activo.

### Aceptar los enlaces cliente servidor

Se encarga de enlazar cada cliente que se quiera conectar al servidor.

In [ ]:
	def aceptarC(self): #Metodo que acepta el enlace entre cliente-servidor.
		print('\nHilo ACEPTAR con ID =',threading.currentThread().getName(), '\n\tHilo en modo DAEMON = ', threading.currentThread().isDaemon(),'\n\tPertenece al PROCESO con PID', os.getpid(), "\n\tHilos activos TOTALES ", threading.active_count()) #imprime que hilos estas utilizando y que id y proceso de cual esta encargado. 
		while True: #Bucle que se mantiene siempre activo
			try: #Try except, para posibles errores y no se detenga el programa.
				conn, addr = self.s.accept() #Acepta el enlace entre cliente-servidor.
				print(f"\nConexion aceptada via {addr}\n") #Imprime el puerto por el que se ha conectado.
				conn.setblocking(False) #Desactivamos el bloqueo, para que no se bloquee mientras este en ejecucion..
				self.clientes.append(conn) #Guardamos el puerto en la pila, para saber cual es el de cada cliente, al momento de mandar los mensajes a cada uno.
				self.imprimirUsuarios()  #Leemos los usuarios que van entrando para enseñarlos por pantalla
			except: pass #Si llega suceder o saltar algun error, ignorarlo sin terminar el problema

### Imprimir usuarios

Metodo que se encarga de imprimir por pantalla los usuarios activos.

In [ ]:
	def imprimirUsuarios(self): #Metodo que imprime por pantalla todos los usuarios activos.
		with open("u22133089AI1UserList.txt", "r") as f: #Una lista de los usuarios en txt, que mantiene los usuarios activos que hay en el chatroom
			print("Clientes conectados ahora [\n" + f.read() + "]") #Imprime por pantalla todos los usuarios.

### Procesar mensaje

Este metodo se encarga de recibir el mensaje que envia un usuario, y no hace nada al menos que haya un minimo de 2 usuarios en el chatRoom, y lo deserealiza/desencripta y llama al metodo broadcast pasandole el mensaje a los demas usuarios menos al que lo envia.

In [ ]:
	def procesarC(self): #Metodo que procesa el mensaje mandado, para deserealizar/descencriptarlo, prepararlo mandar a todos los usuarios menos al que envio el mensaje.
		print('\nHilo PROCESAR con ID =',threading.currentThread().getName(), '\n\tHilo en modo DAEMON = ', threading.currentThread().isDaemon(),'\n\tPertenece al PROCESO con PID', os.getpid(), "\n\tHilos activos TOTALES ", threading.active_count()) #imprime que hilos estas utilizando y que id y proceso de cual esta encargado. 
		while True: #Bucle que se mantiene siempre activo
			if len(self.clientes) > 0: #Tiene que haber al menos 2 usuarios para poder recibir informacion.
				for c in self.clientes: #Pasamo la lista de usuarios como el for va valer siempre al menos 1, aunque se repetira el total de usuarios esten conectados.
					try: #Try except, para posibles errores y no se detenga el programa.
						data = c.recv(1024) #Deserealizar/descencriptar el mensaje, para llamar al metodo broadcaste()
						if data: self.broadcast(data,c) #Llamo al broadcast tantas veces como usuarios haya menos 1, para mandarle la informacion a todos ellos.
					except: pass #Si llega suceder o saltar algun error, ignorarlo sin terminar el problema

### Enviar mensaje

Metodo se encarga de enviar los mensaje a los demas usuarios del chat, opero sin mandarselo al que lo mando, por eso este metodo, pide el puerto (id) del que manda el mensaje, para que cuando en la lista se encuentre no haga nada y pase al siguiente para seguir mandado los menajes.

In [ ]:
	def broadcast(self, msg, cliente): #Metodo que se encarga de mandar el mensaje a cada usuario.
		aux=0 #Variable que utilizaremos para que no imprima tantas veces como usuarios -1 haya.
		for c in self.clientes: #Un bucle que le pasa los puertos y se repite tantos usuarios haya.
			try: #Try except, para posibles errores y no se detenga el programa.
				if c != cliente: #Condicional solo deja pasar, si c es diferente al usuario(cliente) que mando el mensaje.
					if aux==0: #Condicional, para que no imprima cuantos conectados hay y imprimir el mismo mensaje del servidor como tantos usuarios-1 haya.
						print("\nClientes conectados Right now = ", len(self.clientes)) #Te dice el numero de usuarios conectados.
						self.imprimirUsuarios() #Te dice el nombre de los usarios conectados.
						print(pickle.loads(msg)) #Serializa/encripta el mensaje.
						aux=1 #Al valer 1 el auxiliar, hace que no entre al condicional solo 1 vez.
					c.send(msg) #Manda el mensaje a todos los usuarios que no sea el que lo haya enviado.
			except: self.clientes.remove(c) #Si el usuario no esta lo desapila de la lista, para mantener solo los activos.

### Variable que ejecuta el programa

Se declara y inicializa la variable que ejecuta la clase cliente (programa).

In [ ]:
arrancar = Servidor() #Variable que se encarga de correr la clase servidor.

### Resutaldos

    a.  Deberá instanciar 6 clientes y cada uno de ellos debera ingresar mediante el teclado la dirección ip y el puerto, que será los últimos 5 dígitos de su número de expediente. 2 pto.      (__COMPLETADO__)
    b.  Que cada vez que un cliente se conecte, este pueda y deba elegir un nickname que será vuestro correo de la universidad y que deberá ser mostrado cada vez que escriba un texto. 2 pto.   (__COMPLETADO__)
    c.  Que el servidor, imprima y muestre la lista de usuarios (nicknames) conectados al mismo (en ese instante). 2pto.                                                                         (__COMPLETADO__)
    d.  Que el servidor almacene el historial de chat de todos los usuarios conectados durante la sesión y lo guarde en un archivo que se deberá llamar uvuestroExp.txt (u21535220AI1.txt) 4pto. (__COMPLETADO__)

In [ ]:
from IPython import display as dp

In [ ]:
dp.Image("Core1.jpeg").ipynb_checkpoints/

<img src= "https://raw.githubusercontent.com/christianyang22/PCD/master/u22133089AI1ChatRoom/.ipynb_checkpoints/FotoResultado.png?token=GHSAT0AAAAAAB26EYGLMB43MLNPX43D3IWYY3XYR5Q">

## Manual de uso

A continuacion les dare los pasos de como acceder al chat room e iniciar el servidor.

1. Situarnos en la carpeta donde tenemos los archivos Servidor.py.
2. Abrir un terminal, preferible powershell que este en la misma direccion que la carpeta, ahi abriremos el archivo (python ./Servidor.py), lo correremos y le damos el puerto "33089".
3. Nos dara la ip y la guardaremos es donde esta alojado el servidor, que hay que pasarlo a los usuarios que se conectaran a nuestro chatroom.
4. Pasas el IP y puerto (33089) a los usuarios que accedaran al chat room.
5. Cada uno abrira en powershell en la direccion donde se encuentra el archivo Cliente.py.
6. Dentro de la direccion donde se encuentra el archivo se pondra python ./Servidor.py, para correr el programa.
7. Debera ingresar la IP, port(puert) y un nombre de usuario para entrar, si no cumple estos 3 requisistos no podra hacerlo.
8. Luego solo queda de poder disfrutar de chatear entre tus amigos mediante chat room.
9. Cuando se quiera finalizar o cerrar el chat/servidor tendran que darloe al numero 0 y enviarlo, eso cerrara el programa correctamente.

## Conclusion

Hemos aprendido a poder enlazarnos mediante ip y puertos entre computadores, siendo una el servidor los demas los usuarios, ademas de cumplir todos los objetivos, pedido para la entrega.